In [1]:
!pip install -q gradio yfinance tensorflow scikit-learn matplotlib

In [2]:
import gradio as gr
from gradio.themes.soft import Soft

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
import tensorflow as tf # Import tensorflow for the callback
import matplotlib.pyplot as plt
import io
from PIL import Image

# Suppress TensorFlow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

# --- Gradio Progress Callback for Keras ---
# This custom class allows the model's training progress to be displayed in the Gradio UI.
class GradioProgressCallback(tf.keras.callbacks.Callback):
    def __init__(self, progress, epochs, initial_progress=0.2, total_span=0.7):
        super().__init__()
        self.progress = progress
        self.epochs = epochs
        self.initial_progress = initial_progress
        self.total_span = total_span

    def on_epoch_end(self, epoch, logs=None):
        # Calculate current progress within the training span
        training_progress = (epoch + 1) / self.epochs
        # Map it to the overall progress bar
        overall_progress = self.initial_progress + (training_progress * self.total_span)
        self.progress(
            overall_progress,
            desc=f"Analyzing (Training)... Epoch {epoch + 1}/{self.epochs}"
        )

def get_stock_data(ticker, start_date="2012-01-01", end_date="2025-07-27"):
    """Fetches stock data from Yahoo Finance."""
    df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
    return df

def create_dataset(dataset, time_step=1):
    """Creates a dataset for the LSTM model."""
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

def build_and_train_model(X_train, y_train, X_test, y_test, time_step, progress_callback):
    """Builds and trains the LSTM model."""
    model = Sequential([
        Input(shape=(time_step, 1)),
        LSTM(50, return_sequences=True),
        LSTM(50, return_sequences=False),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64, epochs=15, verbose=0, callbacks=[progress_callback])
    return model

def predict_and_plot(ticker, progress=gr.Progress(track_tqdm=True)):
    """The main function that orchestrates the entire process and updates the UI."""
    try:
        progress(0, desc="Starting...")

        progress(0.1, desc=f"Downloading {ticker} dataset...")
        df = get_stock_data(ticker)
        if df.empty:
            return f"Could not fetch data for '{ticker}'. Please check if the ticker symbol is valid.", None
    except Exception as e:
        return f"An error occurred during data download: {e}", None

    progress(0.2, desc="Preprocessing data...")
    df_close = df['Close']
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_close_scaled = scaler.fit_transform(np.array(df_close).reshape(-1, 1))

    training_size = int(len(df_close_scaled) * 0.75)
    train_data, test_data = df_close_scaled[0:training_size, :], df_close_scaled[training_size:len(df_close_scaled), :1]

    time_step = 100
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # The training process will now update the progress from 0.2 to 0.9
    progress_callback = GradioProgressCallback(progress, epochs=15)
    model = build_and_train_model(X_train, y_train, X_test, y_test, time_step, progress_callback)

    progress(0.9, desc="Generating predictions and plot...")

    # Make predictions
    train_predict = model.predict(X_train, verbose=0)
    test_predict = model.predict(X_test, verbose=0)

    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)

    # Plotting
    look_back = time_step
    trainPredictPlot = np.empty_like(df_close_scaled)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict) + look_back, :] = train_predict

    testPredictPlot = np.empty_like(df_close_scaled)
    testPredictPlot[:, :] = np.nan
    test_plot_start_index = training_size + look_back
    testPredictPlot[test_plot_start_index:test_plot_start_index + len(test_predict), :] = test_predict

    plt.style.use('seaborn-v0_8-darkgrid')
    plt.figure(figsize=(15, 8))
    plt.plot(df.index, scaler.inverse_transform(df_close_scaled), color='#007ACC', label='Actual Price')
    plt.plot(df.index, trainPredictPlot, color='#F97306', label='Train Predictions')
    plt.plot(df.index, testPredictPlot, color='#44D7A8', label='Test Predictions')
    plt.title(f'{ticker} Stock Price Prediction', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Close Price USD ($)', fontsize=12)
    plt.legend(loc='upper left')

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close()
    buf.seek(0)
    plot_image = Image.open(buf)

    # Predict the next day's price
    last_100_days = df_close_scaled[-100:]
    last_100_days_reshaped = last_100_days.reshape(1, time_step, 1)
    next_day_price_scaled = model.predict(last_100_days_reshaped, verbose=0)
    next_day_price = scaler.inverse_transform(next_day_price_scaled)[0][0]

    progress(1.0, desc="Showing Prediction")
    return f"The predicted closing price for the next trading day is: **${next_day_price:.2f}**", plot_image

# --- Gradio UI Definition ---
iface = gr.Interface(
    fn=predict_and_plot,
    theme=Soft(),
    inputs=gr.Textbox(lines=1, placeholder="Enter Stock Ticker (e.g., AAPL, GOOG, NVDA)...", label="Stock Ticker"),
    outputs=[
        gr.Markdown(label="Next Day Prediction"),
        gr.Image(label="Prediction Plot", type="pil")
    ],
    title="📊 Stock Price Prediction with LSTM",
    description="Enter a valid stock ticker to train an LSTM model and predict the next day's closing price. The model is trained in real-time with progress updates.",
    examples=[["AAPL"], ["GOOG"], ["NVDA"], ["TSLA"]],
    allow_flagging="never"
)

# Launch the app
if __name__ == "__main__":
    iface.launch(debug = True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://634f6189c98b76e381.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://634f6189c98b76e381.gradio.live
